In [ ]:
!pip install tensorflow --upgrade
!pip install keras --upgrade
!pip install --upgrade tensorflow-metadata



In [ ]:
#importations

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score
import tensorflow as tf


# Loading the training dataset
url = 'https://raw.githubusercontent.com/afrisenti-semeval/afrisent-semeval-2023/main/data/pcm/train.tsv'
df = pd.read_csv(url, sep='\t')

# Extracting the reviews and labels from the training dataset
reviews = df['tweet'].values
labels = df['label'].values

# Spliting the training dataset into training and validation sets
train_reviews, val_reviews, train_labels, val_labels = train_test_split(reviews, labels, test_size=0.2, random_state=42)

# Tokenizing the training reviews
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_reviews)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_reviews)
max_len = max(len(x) for x in sequences)
train_data = pad_sequences(sequences, maxlen=max_len)

# Tokenizing the validation reviews
val_sequences = tokenizer.texts_to_sequences(val_reviews)
val_data = pad_sequences(val_sequences, maxlen=max_len)
# One-hot encode the labels for training and validation sets
train_labels = np.array(pd.get_dummies(train_labels, dummy_na=False, drop_first=False))
val_labels = np.array(pd.get_dummies(val_labels, dummy_na=False, drop_first=False))

# Building the model
model = Sequential()
model.add(Embedding(5000, 32, input_length=max_len))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# Compiling the model with the new metrics
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 
             tf.keras.metrics.Precision(name='precision'), 
             tf.keras.metrics.Recall(name='recall'), 
             tf.keras.metrics.CategoricalAccuracy(name='cat_accuracy'),
             tf.keras.metrics.TruePositives(name='tp'), 
             tf.keras.metrics.FalsePositives(name='fp'), 
             tf.keras.metrics.TrueNegatives(name='tn'), 
             tf.keras.metrics.FalseNegatives(name='fn')])

# Train the model
model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=5, batch_size=64)

# Load the test dataset
test_url = 'https://raw.githubusercontent.com/afrisenti-semeval/afrisent-semeval-2023/main/data/pcm/test.tsv'
test_df = pd.read_csv(test_url, sep='\t')

# Tokenizing the test reviews
test_sequences = tokenizer.texts_to_sequences(test_df['tweet'].values)
test_data = pad_sequences(test_sequences, maxlen=max_len)

# Predicting the labels for the test reviews
predictions = model.predict(test_data)

# Mapping the predicted labels back to the original label names
predicted_labels = np.argmax(predictions, axis=1)
predicted_labels = np.vectorize({0: 'negative', 1: 'neutral', 2: 'positive'}.get)(predicted_labels)



Epoch 1/5
64/64 [==============================] - 4s 22ms/step - loss: 0.7653 - accuracy: 0.6152 - precision: 0.6188 - recall: 0.5557 - cat_accuracy: 0.6152 - tp: 2276.0000 - fp: 1402.0000 - tn: 6790.0000 - fn: 1820.0000 - val_loss: 0.7129 - val_accuracy: 0.6390 - val_precision: 0.6390 - val_recall: 0.6390 - val_cat_accuracy: 0.6390 - val_tp: 655.0000 - val_fp: 370.0000 - val_tn: 1680.0000 - val_fn: 370.0000
Epoch 2/5
64/64 [==============================] - 1s 13ms/step - loss: 0.7158 - accuracy: 0.6265 - precision: 0.6274 - recall: 0.6169 - cat_accuracy: 0.6265 - tp: 2527.0000 - fp: 1501.0000 - tn: 6691.0000 - fn: 1569.0000 - val_loss: 0.6979 - val_accuracy: 0.6390 - val_precision: 0.6390 - val_recall: 0.6390 - val_cat_accuracy: 0.6390 - val_tp: 655.0000 - val_fp: 370.0000 - val_tn: 1680.0000 - val_fn: 370.0000
Epoch 3/5
64/64 [==============================] - 1s 13ms/step - loss: 0.6200 - accuracy: 0.6919 - precision: 0.7000 - recall: 0.6819 - cat_accuracy: 0.6919 - tp: 2793.0000 

In [ ]:

# Predict the labels for the test reviews
predictions = model.predict(test_data)

# Map the predicted labels back to the original label names
predicted_labels = np.argmax(predictions, axis=1)
predicted_labels = np.vectorize({0: 'negative', 1: 'neutral', 2: 'positive'}.get)(predicted_labels)

# Calculate the confusion matrix
cm = confusion_matrix(test_df['label'].values, predicted_labels, labels=['negative', 'neutral', 'positive'])

# Calculate the precision, recall, and F1 scores
report = classification_report(test_df['label'].values, predicted_labels, labels=['negative', 'neutral', 'positive'])


130/130 [==============================] - 0s 3ms/step


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
